## S&P 500 Sector Performance Overview

This notebook loads daily prices for the eleven SPDR sector ETFs (XLC, XLY, XLP, XLE, XLF, XLV, XLI, XLB, XLRE, XLK, XLU), filters the last year of data, and compares their cumulative returns across multiple horizons.

### Data Pipeline
- Download or reuse cached CSVs from `yfinance`, keeping only the `Close` prices per ticker.
- Rename columns so every series shows `Ticker - Sector` in plots and reports.
- Limit the analysis window to the most recent 12 months to highlight current trends.
- Compute simple daily returns with `pct_change()` and roll them into cumulative performance via `(1 + r).cumprod() - 1`.

### Visualization Blocks
1. **Full-year cumulative returns**: compares every sector over the past 12 months.
2. **Last 6 months**: focuses on mid-term rotations and acceleration.
3. **Last 3 months**: captures short-term leadership shifts.

### Key Takeaways
- Identify which sectors lead or lag over each horizon.
- Observe whether leadership is broad-based or concentrated in a few sectors.
- Use the shortened windows to spot emerging trends that may not appear in the full-year chart.

In [1]:
import pandas as pd
import plotly.graph_objects as go

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

import datetime as dt

In [2]:
# portfolio of sector ETFs and their names
ticker_sector_map = {
    "XLC": "Communication Services",
    "XLY": "Consumer Discretionary",
    "XLP": "Consumer Staples",
    "XLE": "Energy",
    "XLF": "Financials",
    "XLV": "Health Care",
    "XLI": "Industrials",
    "XLB": "Materials",
    "XLRE": "Real Estate",
    "XLK": "Technology",
    "XLU": "Utilities",
}

tickers = list(ticker_sector_map.keys())
sector_labels = {ticker: f"{ticker} - {name}" for ticker, name in ticker_sector_map.items()}

In [3]:
start_date = "2023-01-01"
end_date = dt.datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# download data to csv, filename have tickers joined by underscore and end_date
data.to_csv("_".join(tickers) + "_" + end_date + ".csv")

[*********************100%***********************]  11 of 11 completed



In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("_".join(tickers) + "_" + end_date + ".csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# set index as datetime
df.index = pd.to_datetime(df.index)

# keep only level 1 'Close' prices
df = df.xs('Close', level=1, axis=1)

# use descriptive names in charts/legends
df = df.rename(columns=sector_labels)

# Show the result
df.head()

Ticker,XLRE - Real Estate,XLC - Communication Services,XLF - Financials,XLK - Technology,XLI - Industrials,XLV - Health Care,XLU - Utilities,XLB - Materials,XLP - Consumer Staples,XLY - Consumer Discretionary,XLE - Energy
Date,,,,,,,,,,,
2023-01-03,33.786461,47.214489,32.818188,120.799454,94.314232,129.434204,64.770355,73.552422,69.052132,125.403625,76.900299
2023-01-04,34.561169,48.098354,33.344124,121.113068,95.147934,129.778214,65.358337,74.832008,69.321747,127.220360,76.891182
2023-01-05,33.549492,48.166340,33.095501,118.760986,94.122574,128.450043,63.943501,73.647202,68.624435,126.038490,78.294334
2023-01-06,34.524712,48.953091,33.831810,122.240120,96.700356,129.634857,65.229721,76.177948,70.456032,128.998062,79.779503
2023-01-09,34.506485,49.030792,33.707493,123.661163,96.269127,127.465866,65.670708,76.661331,69.768036,129.779434,79.497055


In [5]:
# date filter
# keep only last year of data
one_year_ago = dt.datetime.now() - dt.timedelta(days=365)
df = df[df.index >= one_year_ago]

In [6]:
# df copies for last 6 months and 3 months of data
df_6m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=182))]
df_3m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=91))]

In [7]:
# calculate simple returns with pct_change()
simple_returns = df.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding)
cumprod_simple = (1 + simple_returns).cumprod() - 1

In [8]:
# plot cumprod_simple
fig = go.Figure()
for column in cumprod_simple.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple.index,
            y=cumprod_simple[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last Year)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [9]:
# plot last 6 months only
cumprod_simple_last_6_months = cumprod_simple.last('6M')
fig = go.Figure()
for column in cumprod_simple_last_6_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_6_months.index,
            y=cumprod_simple_last_6_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [10]:
# plot last 3 months only
cumprod_simple_last_3_months = cumprod_simple.last('3M')
fig = go.Figure()
for column in cumprod_simple_last_3_months.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_last_3_months.index,
            y=cumprod_simple_last_3_months[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [11]:
# calculate simple returns with pct_change() for 6m and 3m dataframes
simple_returns_6m = df_6m.pct_change().fillna(0)
simple_returns_3m = df_3m.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding) for 6m and 3m dataframes
cumprod_simple_6m = (1 + simple_returns_6m).cumprod() - 1
cumprod_simple_3m = (1 + simple_returns_3m).cumprod() - 1

In [12]:
# plot last 6 months of cumprod_simple_6m returns 
fig = go.Figure()
for column in cumprod_simple_6m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_6m.index,
            y=cumprod_simple_6m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [13]:
# plot last 3 months of cumprod_simple_3m returns 
fig = go.Figure()
for column in cumprod_simple_3m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_3m.index,
            y=cumprod_simple_3m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="S&P 500 Sectors - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()